In [1]:
# /Users/teliov/TUD/Thesis/Medvice/Notebooks/data/04_06_new_data/data/split
# So we can use the *thesislib* package
import sys
import os

module_path = os.path.abspath("..")

if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import networkx as nx
import json

In [3]:
from thesislib.utils import pathutils

In [10]:
symcat_conditions = pathutils.get_data_file("definitions/symcat_conditions.json")

In [12]:
with open(symcat_conditions) as fp:
    conditions = json.load(fp)

In [28]:
sym_cond_graph = nx.MultiGraph()

In [29]:
from itertools import combinations

In [30]:
for condition_name, condition in conditions.items():
    symptoms = [(val.get("slug"), val.get("probability")) for val in condition.get("symptoms").values()]
    symptoms_combined = combinations(symptoms, 2)
    symptom_edges = [(comb[0][0], comb[1][0], {"condition": condition_name}) for comb in symptoms_combined]
    _ = sym_cond_graph.add_edges_from(symptom_edges)

In [72]:
from copy import deepcopy

In [78]:
import numpy as np

In [73]:
new_conditions = deepcopy(conditions)

In [84]:
TARGET = 5

In [86]:
for condition_name in conditions.keys():
    condition_symptoms = list(conditions.get(condition_name).get("symptoms").keys())
    new_symptom_prob = np.mean([itm.get("probability") for itm in conditions.get(condition_name).get("symptoms").values()])
    new_symptom_prob = int(np.ceil(0.5 * new_symptom_prob))
    
    counter = defaultdict(int)
    for symptom in condition_symptoms:
        connected_symptoms = sym_cond_graph[symptom]
        for connected_symptom, adj_list in connected_symptoms.items():
            counter[connected_symptom] += len(adj_list)
    
    connected_symptoms_list = [(key, value) for key, value in counter.items()]
    connected_symptoms_list = sorted(connected_symptoms_list, key=lambda val: val[1], reverse=True)
    
    symptoms_set = set(condition_symptoms)
    addition = []
    count = 0
    for symp, _ in connected_symptoms_list:
        if symp in symptoms_set:
            continue
        new_conditions.get(condition_name).get("symptoms").update({
            symp: {
                "slug": symp,
                "probability": new_symptom_prob
            }
        })
        count += 1
        if count >= TARGET:
            break

In [88]:
new_conditions_file = pathutils.get_data_file("definitions/symcat_increased_conditions.json")

In [90]:
with open(new_conditions_file, "w") as fp:
    json.dump(new_conditions, fp, indent=4)